# Cygnus X-1 - MCMC samples

In this notebook, we study the raw data of the MCMC sampler of a distribution asymmetric kick neccesary to reproduce current observational estimates of the well-known X-ray binary Cygnus X-1.

This continues what has been shown in the jupyter notebook `1-asb-mcmc`

In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

---

## Import packages

In [ ]:
from pathlib import Path
from typing import Any, Union

import corner
import emcee
import matplotlib.pyplot as plt
import numpy as np
import poskiorb
import yaml

plt.style.use("../config/style.mpl")

## Useful functions

In [ ]:
def load_yaml(fname: Union[str, Path]) -> Any:
    """Load configuration file with YAML format

    Parameters
    ----------
    fname : `str / Path`
        YAML filename

    Returns
    -------
    `yaml.load`
    """

    if isinstance(fname, Path):
        fname = str(fname)

    with open(fname) as f:
        return yaml.load(f, Loader=yaml.FullLoader)

## Load data

In [ ]:
CONFIG_FILENAME = "../config/mcmc-config.yml"
config = load_yaml(fname=CONFIG_FILENAME)

# set some constant values
nwalkers = config["MCMC"].get("walkers")
ndim = config["MCMC"].get("dimension")
nburn = config["MCMC"].get("burn")
nsteps = config["MCMC"].get("steps")
raw_data = "/workdir/cygnusx1/mcmc_corrected_angles.h5"
priorsD = config["MCMC"].get("priorDistributions")
# Cygnus X-1 properties
stellarParameters = config["StellarParameters"]

# load samples & remove burned steps
reader = emcee.backends.HDFBackend(raw_data, read_only=True)
samples = reader.get_chain()

porb = samples[:,:,0].ravel()
m1 = samples[:,:,1].ravel()
m2 = samples[:,:,2].ravel()
w = samples[:,:,3].ravel()
theta = samples[:,:,4].ravel()
phi = samples[:,:,5].ravel()

**Note**: need to correct angles to fit them into their boundaries !

In [ ]:
theta = theta % (2 * np.pi)
theta = np.where(theta < 0, np.pi - (theta % np.pi), theta)
theta = np.where(theta > np.pi, np.pi - (theta % np.pi), theta)

phi = phi % (2 * np.pi)

---

## Results on stellar parameters at core-collapse

In [ ]:
plt.style.use("../config/style.mpl")

fig, axes = plt.subplots(nrows=6, figsize=(10,7), sharex=True)

indices = np.arange(len(porb))

axes[0].plot(indices, porb, color="black", alpha=0.75, lw=0.2)
axes[1].plot(indices, m1, color="black", alpha=0.75, lw=0.2)
axes[2].plot(indices, m2, color="black", alpha=0.75, lw=0.2)
axes[3].plot(indices, w, color="black", alpha=0.75, lw=0.2)
axes[4].plot(indices, theta, color="black", alpha=0.75, lw=0.2)
axes[5].plot(indices, phi, color="black", alpha=0.75, lw=0.2)

axes[5].set_xlabel("step number")
axes[0].set_ylabel("$P_{\\rm orb}$")
axes[1].set_ylabel("$M_1$")
axes[2].set_ylabel("$M_2$")
axes[3].set_ylabel("$w$")
axes[4].set_ylabel("$\\theta$")
axes[5].set_ylabel("$\\phi$")

axes[0].set_xlim([0, len(porb)]);